In [14]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.applications import VGG16

In [15]:
train_dir = './4pose_dataset/train/'
test_dir = './4pose_dataset/test/'
val_dir = './4pose_dataset/val/'

In [16]:
generator = ImageDataGenerator(rescale = 1./255)
train_generator = generator.flow_from_directory(
    directory = train_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
test_generator = generator.flow_from_directory(
    directory = test_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
val_generator = generator.flow_from_directory(
    directory = val_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
print(train_generator.class_indices)

Found 2292 images belonging to 4 classes.
Found 290 images belonging to 4 classes.
Found 284 images belonging to 4 classes.
{'bend': 0, 'headup': 1, 'right': 2, 'waistup': 3}


In [17]:
# 전이학습 -> VGG모델 사용
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (150,150,3)
)

In [18]:
# 가중치 동결 
conv_base.trainable=False

In [19]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [20]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [21]:
model.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

C:\Users\smhrd\AppData\Local\Temp\ipykernel_15044\647605993.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
23/23 [==============================] - 45s 2s/step - loss: 0.8175 - accuracy: 0.6505 - val_loss: 0.5448 - val_accuracy: 0.7782
Epoch 2/20
23/23 [==============================] - 42s 2s/step - loss: 0.4664 - accuracy: 0.8093 - val_loss: 0.4062 - val_accuracy: 0.8486
Epoch 3/20
23/23 [==============================] - 41s 2s/step - loss: 0.3566 - accuracy: 0.8743 - val_loss: 0.3107 - val_accuracy: 0.9049
Epoch 4/20
23/23 [==============================] - 42s 2s/step - loss: 0.2904 - accuracy: 0.9136 - val_loss: 0.2567 - val_accuracy: 0.9225
Epoch 5/20
23/23 [==============================] - 42s 2s/step - loss: 0.2427 - accuracy: 0.9293 - val_loss: 0.2207 - val_accuracy: 0.9366
Epoch 6/20
23/23 [==============================] - 42s 2s/step - loss: 0.2080 - accuracy: 0.9433 - val_loss: 0.1958 - val_accuracy: 0.9401
Epoch 7/20
23/23 [==============================] - 41s 2s/step - loss: 0.1821 - accuracy: 0.9507 - val_loss: 0.1946 - val_accuracy: 0.9261
Epoch 8/20
23/23 [==

In [22]:
# 저장
from keras.models import load_model
model.save('02.VGG_model.h5')